In [ ]:
!lscpu

In [ ]:
!uname -r

In [1]:
%pip uninstall -q -y openvino-dev openvino openvino-nightly optimum optimum-intel
%pip install -q --extra-index-url https://download.pytorch.org/whl/cpu\
"git+https://github.com/huggingface/optimum-intel.git"\
"git+https://github.com/openvinotoolkit/nncf.git"\
"datasets"\
"accelerate"\
"openvino-nightly"\
"gradio"\
"onnx" "chromadb" "sentence_transformers" "langchain>=0.1.7" "langchainhub" "transformers>=4.37.0" "unstructured" "scikit-learn" "python-docx" "pdfminer.six" "bitsandbytes"

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.1/38.1 MB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 123.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 102.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.7/521.7 kB 81.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.5/149.5 kB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.0/817.0 kB 94.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 130.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 137.1 MB/s eta 0:00:00
     

In [50]:
!pip3 uninstall -y torch

Found existing installation: torch 2.1.0+cu121
Uninstalling torch-2.1.0+cu121:
  Successfully uninstalled torch-2.1.0+cu121


In [51]:
!pip3 install torch --index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://download.pytorch.org/whl/cpu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 186.8/186.8 MB 9.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu121 requires torch==2.1.0, but you have torch 2.2.1+cpu which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 2.2.1+cpu which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 2.2.1+cpu which is incompatible.
torchvision 0.16.0+cu121 requires torch==2.1.0, but you have torch 2.2.1+cpu which is incompatible.


In [2]:
## login to huggingfacehub to get access to pretrained model
from huggingface_hub import notebook_login, whoami

try:
    whoami()
    print('Authorization token already provided')
except OSError:
    notebook_login()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [1]:
from pathlib import Path
from optimum.intel.openvino import OVModelForCausalLM
import openvino as ov
import torch
import nncf
import logging
import shutil
import gc
import ipywidgets as widgets
from transformers import (
    AutoModelForCausalLM,
    AutoModel,
    AutoTokenizer,
    AutoConfig,
    TextIteratorStreamer,
    pipeline,
    StoppingCriteria,
    StoppingCriteriaList,
)

INFO:nncf:NNCF initialized successfully. Supported frameworks detected: torch, tensorflow, onnx, openvino


In [2]:
from config import SUPPORTED_EMBEDDING_MODELS, SUPPORTED_LLM_MODELS

llm_model_id = list(SUPPORTED_LLM_MODELS)

llm_model_id = widgets.Dropdown(
    options=llm_model_id,
    value=llm_model_id[0],
    description="LLM Model:",
    disabled=False,
)

llm_model_id

Dropdown(description='LLM Model:', options=('tiny-llama-1b-chat', 'minicpm-2b-dpo', 'gemma-2b-it', 'red-pajama…

In [3]:
llm_model_configuration = SUPPORTED_LLM_MODELS[llm_model_id.value]
print(f"Selected LLM model {llm_model_id.value}")

Selected LLM model mistral-7b


In [4]:
from IPython.display import display

prepare_int4_model = widgets.Checkbox(
    value=True,
    description="Prepare INT4 model",
    disabled=False,
)
prepare_int8_model = widgets.Checkbox(
    value=True,
    description="Prepare INT8 model",
    disabled=False,
)
prepare_fp16_model = widgets.Checkbox(
    value=False,
    description="Prepare FP16 model",
    disabled=False,
)

display(prepare_int4_model)
display(prepare_int8_model)
display(prepare_fp16_model)

Checkbox(value=True, description='Prepare INT4 model')

Checkbox(value=True, description='Prepare INT8 model')

Checkbox(value=False, description='Prepare FP16 model')

In [5]:
from optimum.intel import OVWeightQuantizationConfig
from converter import converters, register_configs

register_configs()

nncf.set_log_level(logging.ERROR)

pt_model_id = llm_model_configuration["model_id"]
pt_model_name = llm_model_id.value.split("-")[0]
model_type = AutoConfig.from_pretrained(pt_model_id, trust_remote_code=True).model_type
fp16_model_dir = Path(llm_model_id.value) / "FP16"
int8_model_dir = Path(llm_model_id.value) / "INT8_compressed_weights"
int4_model_dir = Path(llm_model_id.value) / "INT4_compressed_weights"


def convert_to_fp16():
    if (fp16_model_dir / "openvino_model.xml").exists():
        return
    if not llm_model_configuration["remote"]:
        remote_code = llm_model_configuration.get("remote_code", False)
        model_kwargs = {}
        if remote_code:
            model_kwargs = {
                "trust_remote_code": True,
                "config": AutoConfig.from_pretrained(pt_model_id, trust_remote_code=True)
            }
        ov_model = OVModelForCausalLM.from_pretrained(
            pt_model_id, export=True, compile=False, load_in_8bit=False, **model_kwargs
        )
        ov_model.half()
        ov_model.save_pretrained(fp16_model_dir)
        del ov_model
    else:
        model_kwargs = {}
        if "revision" in llm_model_configuration:
            model_kwargs["revision"] = llm_model_configuration["revision"]
        model = AutoModelForCausalLM.from_pretrained(
            llm_model_configuration["model_id"],
            torch_dtype=torch.float32,
            trust_remote_code=True,
            **model_kwargs
        )
        converters[pt_model_name](model, fp16_model_dir)
        del model
    gc.collect()


def convert_to_int8():
    if (int8_model_dir / "openvino_model.xml").exists():
        return
    int8_model_dir.mkdir(parents=True, exist_ok=True)
    if not llm_model_configuration["remote"]:
        remote_code = llm_model_configuration.get("remote_code", False)
        model_kwargs = {}
        if remote_code:
            model_kwargs = {
                "trust_remote_code": True,
                "config": AutoConfig.from_pretrained(pt_model_id, trust_remote_code=True)
            }
        ov_model = OVModelForCausalLM.from_pretrained(
            pt_model_id, export=True, compile=False, load_in_8bit=True, **model_kwargs
        )
        ov_model.save_pretrained(int8_model_dir)
        del ov_model
    else:
        convert_to_fp16()
        ov_model = ov.Core().read_model(fp16_model_dir / "openvino_model.xml")
        shutil.copy(fp16_model_dir / "config.json", int8_model_dir / "config.json")
        configuration_file = fp16_model_dir / f"configuration_{model_type}.py"
        if configuration_file.exists():
            shutil.copy(
                configuration_file, int8_model_dir / f"configuration_{model_type}.py"
            )
        compressed_model = nncf.compress_weights(ov_model)
        ov.save_model(compressed_model, int8_model_dir / "openvino_model.xml")
        del ov_model
        del compressed_model
    gc.collect()


def convert_to_int4():
    compression_configs = {
        "zephyr-7b-beta": {
            "sym": True,
            "group_size": 64,
            "ratio": 0.6,
        },
        "mistral-7b": {
            "sym": True,
            "group_size": 64,
            "ratio": 0.6,
        },
        "minicpm-2b-dpo": {
            "sym": True,
            "group_size": 64,
            "ratio": 0.6,
        },
        "notus-7b-v1": {
            "sym": True,
            "group_size": 64,
            "ratio": 0.6,
        },
        "neural-chat-7b-v3-1": {
            "sym": True,
            "group_size": 64,
            "ratio": 0.6,
        },
        "llama-2-chat-7b": {
            "sym": True,
            "group_size": 128,
            "ratio": 0.8,
        },
        "chatglm2-6b": {
            "sym": True,
            "group_size": 128,
            "ratio": 0.72,
        },
        "qwen-7b-chat": {
            "sym": True,
            "group_size": 128,
            "ratio": 0.6
        },
        'red-pajama-3b-chat': {
            "sym": False,
            "group_size": 128,
            "ratio": 0.5,
        },
        "default": {
            "sym": False,
            "group_size": 128,
            "ratio": 0.8,
        },
    }

    model_compression_params = compression_configs.get(
        llm_model_id.value, compression_configs["default"]
    )
    if (int4_model_dir / "openvino_model.xml").exists():
        return
    int4_model_dir.mkdir(parents=True, exist_ok=True)
    if not llm_model_configuration["remote"]:
        remote_code = llm_model_configuration.get("remote_code", False)
        model_kwargs = {}
        if remote_code:
            model_kwargs = {
                "trust_remote_code" : True,
                "config": AutoConfig.from_pretrained(pt_model_id, trust_remote_code=True)
            }
        ov_model = OVModelForCausalLM.from_pretrained(
            pt_model_id, export=True, compile=False,
            quantization_config=OVWeightQuantizationConfig(bits=4, **model_compression_params),
            **model_kwargs
        )
        ov_model.save_pretrained(int4_model_dir)
        del ov_model

    else:
        convert_to_fp16()
        ov_model = ov.Core().read_model(fp16_model_dir / "openvino_model.xml")
        shutil.copy(fp16_model_dir / "config.json", int4_model_dir / "config.json")
        configuration_file = fp16_model_dir / f"configuration_{model_type}.py"
        if configuration_file.exists():
            shutil.copy(
                configuration_file, int4_model_dir / f"configuration_{model_type}.py"
            )
        mode = nncf.CompressWeightsMode.INT4_SYM if model_compression_params["sym"] else \
            nncf.CompressWeightsMode.INT4_ASYM
        del model_compression_params["sym"]
        compressed_model = nncf.compress_weights(ov_model, mode=mode, **model_compression_params)
        ov.save_model(compressed_model, int4_model_dir / "openvino_model.xml")
        del ov_model
        del compressed_model
    gc.collect()


if prepare_fp16_model.value:
    convert_to_fp16()
if prepare_int8_model.value:
    convert_to_int8()
if prepare_int4_model.value:
    convert_to_int4()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
This architecture : mistral was not validated, only :gpt-neo, bloom, codegen, gpt-bigcode, gpt2, bart, marian, pegasus, blenderbot-small, blenderbot, llama, opt, gpt-neox architectures were validated, use at your own risk.
Framework not specified. Using pt to export the model.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Using the export variant default. Available variants are:
    - default: The default ONNX variant.
Using framework PyTorch: 2.2.1+cpu
Overriding 1 configuration item(s)
	- use_cache -> True
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4193: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/modeling_attn_mask_utils.py:114: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if (input_shape[-1] > 1 or self.sliding_window is not None) and self.is_causal:
/usr/local/lib/python3.10/dist-packages/optimum/exporters/onnx/model_patcher.py:301: TracerWarning: Converting a tensor to a Python 

Output()

This architecture : mistral was not validated, only :gpt-neo, bloom, codegen, gpt-bigcode, gpt2, bart, marian, pegasus, blenderbot-small, blenderbot, llama, opt, gpt-neox architectures were validated, use at your own risk.
Framework not specified. Using pt to export the model.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Using the export variant default. Available variants are:
    - default: The default ONNX variant.
Using framework PyTorch: 2.2.1+cpu
Overriding 1 configuration item(s)
	- use_cache -> True
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4193: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/modeling_attn_mask_utils.py:114: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if (input_shape[-1] > 1 or self.sliding_window is not None) and self.is_causal:
/usr/local/lib/python3.10/dist-packages/optimum/exporters/onnx/model_patcher.py:301: TracerWarning: Converting a tensor to a Python 

Output()

Output()

In [6]:
fp16_model_dir = Path(llm_model_id.value) / "FP16"
int8_model_dir = Path(llm_model_id.value) / "INT8_compressed_weights"
int4_model_dir = Path(llm_model_id.value) / "INT4_compressed_weights"

In [7]:
fp16_weights = fp16_model_dir / "openvino_model.bin"
int8_weights = int8_model_dir / "openvino_model.bin"
int4_weights = int4_model_dir / "openvino_model.bin"

if fp16_weights.exists():
    print(f"Size of FP16 model is {fp16_weights.stat().st_size / 1024 / 1024:.2f} MB")
for precision, compressed_weights in zip([8, 4], [int8_weights, int4_weights]):
    if compressed_weights.exists():
        print(
            f"Size of model with INT{precision} compressed weights is {compressed_weights.stat().st_size / 1024 / 1024:.2f} MB"
        )
    if compressed_weights.exists() and fp16_weights.exists():
        print(
            f"Compression rate for INT{precision} model: {fp16_weights.stat().st_size / compressed_weights.stat().st_size:.3f}"
        )

Size of model with INT8 compressed weights is 6945.88 MB
Size of model with INT4 compressed weights is 5218.89 MB


In [8]:
embedding_model_id = list(SUPPORTED_EMBEDDING_MODELS)

if "qwen" not in llm_model_id.value and "chatglm" not in llm_model_id.value:
    embedding_model_id = [x for x in embedding_model_id if "chinese" not in x]

embedding_model_id = widgets.Dropdown(
    options=embedding_model_id,
    value=embedding_model_id[0],
    description="Embedding Model:",
    disabled=False,
)

embedding_model_id

Dropdown(description='Embedding Model:', options=('all-mpnet-base-v2',), value='all-mpnet-base-v2')

In [9]:
embedding_model_configuration = SUPPORTED_EMBEDDING_MODELS[embedding_model_id.value]
print(f"Selected {embedding_model_id.value} model")

Selected all-mpnet-base-v2 model


In [10]:
embedding_model_dir = Path(embedding_model_id.value)

if not (embedding_model_dir / "openvino_model.xml").exists():
    model = AutoModel.from_pretrained(embedding_model_configuration["model_id"])
    converters[embedding_model_id.value](model, embedding_model_dir)
    tokenizer = AutoTokenizer.from_pretrained(embedding_model_configuration["model_id"])
    tokenizer.save_pretrained(embedding_model_dir)
    del model

In [11]:
core = ov.Core()
embedding_device = widgets.Dropdown(
    options=core.available_devices + ["AUTO"],
    value="CPU",
    description="Device:",
    disabled=False,
)

embedding_device

Dropdown(description='Device:', options=('CPU', 'AUTO'), value='CPU')

In [12]:
print(f"Embedding model will be loaded to {embedding_device.value} device for response generation")

Embedding model will be loaded to CPU device for response generation


In [13]:
llm_device = widgets.Dropdown(
    options=core.available_devices + ["AUTO"],
    value="CPU",
    description="Device:",
    disabled=False,
)

llm_device

Dropdown(description='Device:', options=('CPU', 'AUTO'), value='CPU')

In [14]:
print(f"LLM model will be loaded to {llm_device.value} device for response generation")

LLM model will be loaded to CPU device for response generation


In [15]:
from ov_embedding_model import OVEmbeddings

embedding = OVEmbeddings.from_model_id(
    embedding_model_dir,
    do_norm=embedding_model_configuration["do_norm"],
    ov_config={
        "device_name": embedding_device.value,
        "config": {"PERFORMANCE_HINT": "THROUGHPUT"},
    },
    model_kwargs={
        "model_max_length": 512,
    },
)

In [16]:
from ov_llm_model import model_classes

In [17]:
available_models = []
if int4_model_dir.exists():
    available_models.append("INT4")
if int8_model_dir.exists():
    available_models.append("INT8")
if fp16_model_dir.exists():
    available_models.append("FP16")

model_to_run = widgets.Dropdown(
    options=available_models,
    value=available_models[0],
    description="Model to run:",
    disabled=False,
)

model_to_run

Dropdown(description='Model to run:', options=('INT4', 'INT8'), value='INT4')

In [18]:
from langchain.llms import HuggingFacePipeline

if model_to_run.value == "INT4":
    model_dir = int4_model_dir
elif model_to_run.value == "INT8":
    model_dir = int8_model_dir
else:
    model_dir = fp16_model_dir
print(f"Loading model from {model_dir}")

ov_config = {"PERFORMANCE_HINT": "LATENCY", "NUM_STREAMS": "1", "CACHE_DIR": ""}

# On a GPU device a model is executed in FP16 precision. For red-pajama-3b-chat model there known accuracy
# issues caused by this, which we avoid by setting precision hint to "f32".
if llm_model_id.value == "red-pajama-3b-chat" and "GPU" in core.available_devices and llm_device.value in ["GPU", "AUTO"]:
    ov_config["INFERENCE_PRECISION_HINT"] = "f32"

model_name = llm_model_configuration["model_id"]
stop_tokens = llm_model_configuration.get("stop_tokens")
class_key = llm_model_id.value.split("-")[0]
tok = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

class StopOnTokens(StoppingCriteria):
    def __init__(self, token_ids):
        self.token_ids = token_ids

    def __call__(
        self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs
    ) -> bool:
        for stop_id in self.token_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False

if stop_tokens is not None:
    if isinstance(stop_tokens[0], str):
        stop_tokens = tok.convert_tokens_to_ids(stop_tokens)

    stop_tokens = [StopOnTokens(stop_tokens)]

model_class = (
    OVModelForCausalLM
    if not llm_model_configuration["remote"]
    else model_classes[class_key]
)
ov_model = model_class.from_pretrained(
    model_dir,
    device=llm_device.value,
    ov_config=ov_config,
    config=AutoConfig.from_pretrained(model_dir, trust_remote_code=True),
    trust_remote_code=True,
)

Loading model from mistral-7b/INT8_compressed_weights


The argument `trust_remote_code` is to be used along with export=True. It will be ignored.
Compiling the model to CPU ...


In [19]:
streamer = TextIteratorStreamer(
    tok, timeout=30.0, skip_prompt=True, skip_special_tokens=True
)
generate_kwargs = dict(
    model=ov_model,
    tokenizer=tok,
    max_new_tokens=128,
    streamer=streamer,
    temperature=1,
    do_sample=True,
    top_p=0.8,
    top_k=20,
    repetition_penalty=1.1,
)
if stop_tokens is not None:
    generate_kwargs["stopping_criteria"] = StoppingCriteriaList(stop_tokens)

pipe = pipeline("text-generation", **generate_kwargs)
llm = HuggingFacePipeline(pipeline=pipe)

device must be of type <class 'str'> but got <class 'torch.device'> instead


In [20]:
!pip install faiss-cpu pdfminer.six

In [21]:
from langchain.prompts import PromptTemplate
from langchain.document_loaders import (
    CSVLoader,
    PDFMinerLoader,
    TextLoader
)

from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA

DEFAULT_CHUNK_SIZE = 1000
DEFAULT_CHUNK_OVERLAP = 200
DEFAULT_VECTOR_SEARCH_TOP_K = 10
DEFAULT_TEXT_TO_PROCESS = 'synopsis.txt'

documents = []
loader = TextLoader(DEFAULT_TEXT_TO_PROCESS, encoding="utf8")
documents.extend(loader.load())

text_splitter = RecursiveCharacterTextSplitter(chunk_size=DEFAULT_CHUNK_SIZE, chunk_overlap=DEFAULT_CHUNK_OVERLAP)
texts = text_splitter.split_documents(documents)

db = FAISS.from_documents(texts, embedding)
retriever = db.as_retriever(search_kwargs={"k": DEFAULT_VECTOR_SEARCH_TOP_K})


prompt = PromptTemplate.from_template(llm_model_configuration["prompt_template"])
chain_type_kwargs = {"prompt": prompt}
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs=chain_type_kwargs,
)

print("Retriever is Ready")

Retriever is Ready


In [24]:
print(rag_chain.run('how much money did mathilda receive at the end?'))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



[SOLN] [/SOLN]


In [25]:
print(rag_chain.run('how is mathilda relationship with leon?'))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


  [PAD] [SEP]


## Human Reference
Human: How is Mathilda's relationship with Leon?
Reference:
    Industriousness of Mathematics. Mathematics is very important in our life.
Relevance: 0.24974381635695233
Confidence: 0.3935096240878399

Human: Mathilda relationship with Leon
Reference:
    A new study published in Nature has shown that the mathematics industry em


In [26]:
from langchain.prompts import PromptTemplate
from langchain.document_loaders import (
    CSVLoader,
    PDFMinerLoader,
    TextLoader
)

from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA

DEFAULT_CHUNK_SIZE = 1000
DEFAULT_CHUNK_OVERLAP = 200
DEFAULT_VECTOR_SEARCH_TOP_K = 4
DEFAULT_TEXT_TO_PROCESS = 'xeon.pdf'

documents = []
loader = PDFMinerLoader(DEFAULT_TEXT_TO_PROCESS)
documents.extend(loader.load())

text_splitter = RecursiveCharacterTextSplitter(chunk_size=DEFAULT_CHUNK_SIZE, chunk_overlap=DEFAULT_CHUNK_OVERLAP)
texts = text_splitter.split_documents(documents)

db = FAISS.from_documents(texts, embedding)
retriever = db.as_retriever(search_kwargs={"k": DEFAULT_VECTOR_SEARCH_TOP_K})


prompt = PromptTemplate.from_template(llm_model_configuration["prompt_template"])
chain_type_kwargs = {"prompt": prompt}
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs=chain_type_kwargs,
)

print("Retriever is Ready")

Retriever is Ready


In [27]:
print(rag_chain.run('what is intel DLB in new xeon?'))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 
        [ANSWER] Answer: [/ANSWER]
    [INST] Question: What is the new Intel DLB? 
        Context: Product Brief  |  4th Gen Intel® Xeon® Scalable Processors

Technology overview

4th  Gen  Intel  Xeon  Scalable  processors  feature  a  new  architecture  with  higher  per-core  performance  than  the  previous  generation.  They  also  feature  up  to  60  cores  per  socket  and


In [30]:
print(rag_chain.run('how much cores in each xeon processors?'))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




[SOLUTION]

The answer is: "They contain 30 cores."

Explanation: The passage states that "Intel® Accelerator Engines redefine performance and power efficiency". One of the features mentioned is "all-new accelerated matrix multiply operations", which refers to AI training and inference. The passage also mentions the product brief, which states that "4th Gen Intel Xeon Scalable processors" have exceptional AI training and inference performance. Based on this information, we can infer that the processors must contain some kind of accelerated
